This notebook uses the kdot accidents layer and AADT from the KDOT Khub publication database to calaulate crash rates using the equation

$$R = \frac{C x 100,000,000}{V x 365 x N x L}$$

The variables in this equation are:

R = crash rate for the road segment expressed as crashes per 100 million vehicle-miles of travel,

C = Total number of crashes in the study period

V = Traffic volumes using Average Annual Daily Traffic (AADT) volumes

N = Number of years of data

L = Length of the roadway segment in miles

https://safety.fhwa.dot.gov/local_rural/training/fhwasa1109/app_c.cfm

https://wfs.ksdot.org/arcgis_web_adaptor/rest/services/Transportation/Accidents/FeatureServer



In [2]:
projdb =r"C:\Users\kgonterwitz\Documents\ArcGIS\Projects\KDOT_CrashMaps\KDOT_CrashMaps.gdb"

In [ ]:
#copy local Accidents data from KDOT Wfs for Douglas County to project database

with arcpy.EnvManager(scratchWorkspace=projdb, outputCoordinateSystem="PROJCS['NAD_1983_2011_KS_RCS_Zone_11',GEOGCS['GCS_NAD_1983_2011',DATUM['D_NAD_1983_2011',SPHEROID['GRS_1980',6378137.0,298.257222101]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]],PROJECTION['Lambert_Conformal_Conic'],PARAMETER['False_Easting',11500000.0],PARAMETER['False_Northing',600000.0],PARAMETER['Central_Meridian',-95.25],PARAMETER['Standard_Parallel_1',39.1],PARAMETER['Scale_Factor',1.000033],PARAMETER['Latitude_Of_Origin',39.1],UNIT['Foot_US',0.3048006096012192]]", workspace=projdb):
    arcpy.conversion.FeatureClassToFeatureClass("Accidents", projdb, "Accidents", "ACC_COUNTY = 'DOUGLAS'", '#', '')

In [8]:
#copy local AADT data from Khub publiation geodatabase to project database 
#selected are assigned routes to douglas county and routes longer than ten feet
#the minimum length threshold should be reviewed and possible raised
#if the minimum lenth threshod rises above 30 feet, raise spatial join parameter accordingly

arcpy.management.SelectLayerByAttribute("ev_AADT", "NEW_SELECTION", "RouteID LIKE '023%' And Shape_Length > 35", None)

with arcpy.EnvManager(outputCoordinateSystem="PROJCS['NAD_1983_2011_KS_RCS_Zone_11',GEOGCS['GCS_NAD_1983_2011',DATUM['D_NAD_1983_2011',SPHEROID['GRS_1980',6378137.0,298.257222101]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]],PROJECTION['Lambert_Conformal_Conic'],PARAMETER['False_Easting',11500000.0],PARAMETER['False_Northing',600000.0],PARAMETER['Central_Meridian',-95.25],PARAMETER['Standard_Parallel_1',39.1],PARAMETER['Scale_Factor',1.000033],PARAMETER['Latitude_Of_Origin',39.1],UNIT['Foot_US',0.3048006096012192]]"):
    arcpy.conversion.FeatureClassToFeatureClass("ev_AADT", projdb, "AADT_2019", "RouteID LIKE '023%'", '#', '')

In [9]:
#this cell outputs the calculated crash rate for mapping and review

arcpy.analysis.SpatialJoin("AADT_2019", "Accidents_DG", "SJ_Acc_AADT_2019", "JOIN_ONE_TO_ONE", "KEEP_ALL", '#', "INTERSECT", "36 Feet", '')
arcpy.management.CalculateField("SJ_Acc_AADT_2019", "CrashRateVolumetric", "!Join_Count! * 100000000/(!AADTCount! * 365 * 10 * (!ToMeasure!-!FromMeasure!))", "PYTHON3", '', "DOUBLE")

<Result 'SJ_Acc_AADT_2019'>

In [10]:
#pedestrian (and bicycle) related crashes included

arcpy.management.MakeFeatureLayer("Accidents_DG", "Accidents_BikePed", "ALL_PEDESTRIANS > 0 Or PEDESTRIAN_ACCS > 0 Or PEDAL_CYCLIST_ACCS > 0", None, "#")

arcpy.analysis.SpatialJoin("AADT_2019", "Accidents_BikePed", "SJ_AccBP_AADT_2019", "JOIN_ONE_TO_ONE", "KEEP_ALL", '#', "INTERSECT", "36 Feet", '')

arcpy.management.CalculateField("SJ_AccBP_AADT_2019", "PedCrashRateVolumetric", "!Join_Count! * 100000000/(!AADTCount! * 365 * 10 * (!ToMeasure!-!FromMeasure!))", "PYTHON3", '', "DOUBLE")

<Result 'SJ_AccBP_AADT_2019'>

In [62]:
arcpy.management.SelectLayerByAttribute("Accidents", "NEW_SELECTION", "ACC_COUNTY = 'DOUGLAS'", None)
arcpy.lr.LocateFeaturesAlongRoutes("Accidents", "LRS_County_Primary", "RouteId", "24 Feet", r"C:\Users\kgonterwitz\Documents\ArcGIS\Projects\KDOT_CrashMaps\KDOT_CrashMaps.gdb\Accidents_LFAR", "RouteId Point MEAS", "ALL", "DISTANCE", "ZERO", "FIELDS", "M_DIRECTON")


id,value
0,C:\Users\kgonterwitz\Documents\ArcGIS\Projects\KDOT_CrashMaps\KDOT_CrashMaps.gdb\Accidents_LFAR
1,RouteId Point MEAS
